# Interactive Report Assistant — Google Docs + Gemini

Run the next cell to set up the assistant. Then follow on-screen buttons.

In [ ]:

# bootstrap: fetch or define the assistant logic inline (no external fetch needed)
# You can expand this cell if you'd like to inspect or customize anything.

GEMINI_MODEL = "gemini-2.5-flash"
GEMINI_API_KEY = None
CLIENT_SECRETS_JSON = None
DEFAULT_TOPIC = "GenAI strategy for customer support"
DEFAULT_TITLE = "Executive Report: GenAI Strategy for Support"

import os, json, base64, re, uuid, datetime
import ipywidgets as widgets
from IPython.display import display, Markdown

SCOPES = [
    "https://www.googleapis.com/auth/documents",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive.metadata.readonly"
]

def google_auth(creds_path: str=None):
    try:
        from google.colab import auth as colab_auth  # type: ignore
        colab_auth.authenticate_user()
        from googleapiclient.discovery import build
        docs  = build("docs", "v1")
        drive = build("drive", "v3")
        return docs, drive
    except Exception:
        from google_auth_oauthlib.flow import InstalledAppFlow
        from google.auth.transport.requests import Request
        from google.oauth2.credentials import Credentials
        from googleapiclient.discovery import build
        token_file = "/mnt/data/token.json"
        creds = None
        if os.path.exists(token_file):
            creds = Credentials.from_authorized_user_file(token_file, SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                if not creds_path or not os.path.exists(creds_path):
                    raise RuntimeError("CLIENT_SECRETS_JSON not set or file missing.")
                flow = InstalledAppFlow.from_client_secrets_file(creds_path, SCOPES)
                creds = flow.run_local_server(port=0)
            with open(token_file, "w") as f:
                f.write(creds.to_json())
        docs  = build("docs", "v1", credentials=creds)
        drive = build("drive", "v3", credentials=creds)
        return docs, drive

def parse_gemini_json(resp):
    if getattr(resp, "text", None):
        t = (resp.text or "").strip()
        if t:
            try:
                return json.loads(t)
            except Exception:
                pass
    for c in getattr(resp, "candidates", []) or []:
        content = getattr(c, "content", None)
        parts = getattr(content, "parts", []) if content else []
        for p in parts:
            if getattr(p, "text", None):
                t = p.text.strip()
                if t:
                    try:
                        return json.loads(t)
                    except Exception:
                        m = re.search(r"\{[\s\S]*\}\s*$", t)
                        if m:
                            return json.loads(m.group(0))
            if hasattr(p, "inline_data"):
                mime = getattr(p.inline_data, "mime_type", "")
                data = getattr(p, "inline_data").get("data", "") if isinstance(p, dict) else getattr(p.inline_data, "data", "")
                if mime == "application/json" and data:
                    decoded = base64.b64decode(data).decode("utf-8")
                    return json.loads(decoded)
    raise RuntimeError("No JSON found in Gemini response.")

def ensure_gemini_model():
    import google.generativeai as genai
    api_key = os.environ.get("GEMINI_API_KEY") or GEMINI_API_KEY
    if not api_key:
        raise RuntimeError("Set GEMINI_API_KEY.")
    genai.configure(api_key=api_key)
    return genai.GenerativeModel(
        GEMINI_MODEL,
        generation_config={
            "response_mime_type": "application/json",
            "temperature": 0.2,
            "max_output_tokens": 1536,
        },
    )

from googleapiclient.errors import HttpError
class GDocsHelper:
    def __init__(self, docs, drive, log_path):
        self.docs = docs; self.drive = drive; self.log_path = log_path
    def _log(self, e):
        e["ts"] = datetime.datetime.utcnow().isoformat()+"Z"
        os.makedirs(os.path.dirname(self.log_path), exist_ok=True)
        with open(self.log_path, "a") as f: f.write(json.dumps(e)+"\n")
    def create_doc(self, title):
        d = self.docs.documents().create(body={"title": title}).execute()
        doc_id = d["documentId"]; link = f"https://docs.google.com/document/d/{doc_id}/edit"
        self._log({"event":"create_doc","title":title,"docId":doc_id}); return doc_id, link
    def append_text(self, doc_id, text):
        reqs=[{"insertText":{"text":text,"endOfSegmentLocation":{}}}]
        self.docs.documents().batchUpdate(documentId=doc_id, body={"requests": reqs}).execute()
        self._log({"event":"append_text","docId":doc_id,"chars":len(text)})
    def replace_all(self, doc_id, query, replace, match_case=False):
        reqs=[{"replaceAllText":{"containsText":{"text":query,"matchCase":match_case},"replaceText":replace}}]
        self.docs.documents().batchUpdate(documentId=doc_id, body={"requests": reqs}).execute()
        self._log({"event":"replace_all","docId":doc_id,"query":query,"replace_len":len(replace)})
    def insert_toc(self, doc_id):
        reqs=[{"insertTableOfContents":{"location":{"index":1},"tableOfContents":{"suggestedInsertionIds":[]}}}]
        self.docs.documents().batchUpdate(documentId=doc_id, body={"requests": reqs}).execute()
        self._log({"event":"insert_toc","docId":doc_id})
    def insert_image(self, doc_id, url, width_pts=360):
        reqs=[{"insertInlineImage":{"location":{"endOfSegmentLocation":{}},"uri":url,"objectSize":{"height":{"magnitude":width_pts*0.5625,"unit":"PT"},"width":{"magnitude":width_pts,"unit":"PT"}}}}]
        self.docs.documents().batchUpdate(documentId=doc_id, body={"requests": reqs}).execute()
        self._log({"event":"insert_image","docId":doc_id,"url":url})
    def add_comment(self, file_id, content):
        self.drive.comments().create(fileId=file_id, body={"content": content}).execute()
        self._log({"event":"add_comment","fileId":file_id})

def plan_actions(user_msg, topic=None):
    try:
        model = ensure_gemini_model()
        schema = {
            "instruction": "Return STRICT JSON: {\"actions\":[{\"type\":one_of[create_doc,append_text,replace_all,insert_toc,insert_image,add_comment],\"args\":{}}]}",
            "topic": topic,
            "user_request": user_msg
        }
        resp = model.generate_content(json.dumps(schema))
        js = parse_gemini_json(resp)
        if "actions" in js and isinstance(js["actions"], list) and js["actions"]:
            return js
        raise RuntimeError("No actions returned.")
    except Exception as e:
        return {"actions":[{"type":"append_text","args":{"text": "\n\n"+user_msg}}], "_note": f"fallback: {e}"}}

def execute_actions(helper, actions, state):
    out=[]; doc_id=state.get("doc_id"); file_id=state.get("file_id"); link=state.get("link")
    for a in actions.get("actions", []):
        t=a.get("type"); args=a.get("args", {})
        try:
            if t=="create_doc":
                title=args.get("title") or state.get("title") or "Untitled Report"
                doc_id, link = helper.create_doc(title); file_id = doc_id; state.update({"doc_id":doc_id,"file_id":file_id,"link":link,"title":title})
                out.append(f"- create_doc ✓ → [{title}]({link})")
            elif t=="append_text":
                if not doc_id:
                    title=state.get("title") or "Untitled Report"; doc_id,link=helper.create_doc(title); file_id=doc_id; state.update({"doc_id":doc_id,"file_id":file_id,"link":link}); out.append(f"- (auto) create_doc ✓ → [{title}]({link})")
                helper.append_text(doc_id, args.get("text","")); out.append("- append_text ✓")
            elif t=="replace_all":
                if not doc_id: out.append("- replace_all ✗ (no doc)"); continue
                helper.replace_all(doc_id, args.get("query",""), args.get("replace",""), bool(args.get("match_case", False))); out.append("- replace_all ✓")
            elif t=="insert_toc":
                if not doc_id: out.append("- insert_toc ✗ (no doc)"); continue
                helper.insert_toc(doc_id); out.append("- insert_toc ✓")
            elif t=="insert_image":
                if not doc_id: out.append("- insert_image ✗ (no doc)"); continue
                helper.insert_image(doc_id, args.get("url","")); out.append("- insert_image ✓")
            elif t=="add_comment":
                if not file_id: out.append("- add_comment ✗ (no file_id)"); continue
                helper.add_comment(file_id, args.get("content","")); out.append("- add_comment ✓")
            else:
                out.append(f"- (skip) unknown action: {t}")
        except Exception as e:
            out.append(f"- {t} ✗ error: {e}")
    return out, state

RUN_ID = f"gdocs-{uuid.uuid4().hex[:8]}"; LOG_PATH = f"/mnt/data/{RUN_ID}_actions.jsonl"
_state={"docs":None,"drive":None,"helper":None,"doc_id":None,"file_id":None,"link":None,"title":DEFAULT_TITLE,"topic":DEFAULT_TOPIC}

topic_in   = widgets.Text(value=DEFAULT_TOPIC, description="Topic")
title_in   = widgets.Text(value=DEFAULT_TITLE, description="Doc title")
connect_btn= widgets.Button(description="1) Connect Google", button_style="info")
create_btn = widgets.Button(description="2) Create & Draft Report", button_style="success")
chat_box   = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', min_height='160px'))
input_box  = widgets.Text(placeholder="Type an edit request and press Enter…")
send_btn   = widgets.Button(description="Send")
doc_link   = widgets.HTML(value="")
logs_label = widgets.HTML(value="<b>Activity Log</b>")

history=[]

def render_chat():
    chat_box.clear_output()
    with chat_box:
        for u,a in history:
            display(Markdown(f"**You:** {u}")); display(Markdown(a))

def on_connect(_):
    try:
        docs,drive=google_auth(CLIENT_SECRETS_JSON); _state["docs"],_state["drive"]=docs,drive; _state["helper"]=GDocsHelper(docs,drive,LOG_PATH)
        doc_link.value="<span style='color:green;'>Connected to Google APIs ✓</span>"
    except Exception as e:
        doc_link.value=f"<span style='color:red;'>Auth error: {e}</span>"

def initial_report_for_topic(topic):
    return f"""# {topic}

## Executive Summary
- Objectives
- Key outcomes
- Scope and constraints

## Background
- Context and stakeholders
- Current state & pain points

## Opportunities
- Quick wins (30–60 days)
- Strategic bets (6–12 months)

## Risks & Mitigations
- Adoption, security, cost

## Recommendations
- Roadmap
- KPIs and next steps
"""

def on_create(_):
    topic=topic_in.value.strip() or "Untitled Topic"
    title=title_in.value.strip() or f"Executive Report — {topic}"
    _state["topic"],_state["title"]=topic,title
    if not _state.get("helper"):
        doc_link.value="<span style='color:red;'>Connect to Google first.</span>"; return
    try:
        doc_id,link=_state["helper"].create_doc(title)
        _state.update({"doc_id":doc_id,"file_id":doc_id,"link":link})
        _state["helper"].insert_toc(doc_id)
        _state["helper"].append_text(doc_id, initial_report_for_topic(topic))
        doc_link.value=f"<a href='{link}' target='_blank'>Open Google Doc</a>"
        bot_msg=f"Created doc → [{title}]({link}). Inserted TOC and starter outline."
        history.append(("Create report", bot_msg)); render_chat()
    except Exception as e:
        doc_link.value=f"<span style='color:red;'>Create error: {e}</span>"

def on_send(_=None):
    user_msg=input_box.value.strip()
    if not user_msg: return
    input_box.value=""
    try:
        plan=plan_actions(user_msg, topic=_state.get("topic")); actions_md="```json\n"+json.dumps(plan, indent=2)+"\n```"
    except Exception as e:
        plan={"actions":[{"type":"append_text","args":{"text":"\n\n"+user_msg}}],"_note":str(e)}; actions_md=f"(planner error) {e}"
    helper=_state.get("helper")
    if not helper:
        history.append((user_msg, "Connect to Google first.")); render_chat(); return
    transcript,_=execute_actions(helper, plan, _state)
    final_link=_state.get("link"); summary="\n".join(transcript)
    if final_link: summary += f"\n\n[Open Doc]({final_link})"
    bot_msg="**Planned actions**\n"+actions_md+"\n\n**Execution**\n"+summary
    history.append((user_msg, bot_msg)); render_chat()

connect_btn.on_click(on_connect); create_btn.on_click(on_create); send_btn.on_click(on_send); input_box.on_submit(on_send)

display(widgets.VBox([
    widgets.HBox([topic_in, title_in]),
    widgets.HBox([connect_btn, create_btn]),
    doc_link,
    logs_label,
    chat_box,
    widgets.HBox([input_box, send_btn])
]))
render_chat()
